In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
#Video Feed
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    cv2.imshow('Mediapipe Feed',frame)

    if cv2.waitKey(10) & 0XFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Making Detections

In [4]:
cap = cv2.VideoCapture(0)
# Setting up Media Instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        #Recolour the image to RGB to make it compatible with Mediapipeqqqq
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make Detection
        results = pose.process(image)

        #Recolor back to BGR to make it compatible with Open CV
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Render Detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Feed',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
        

Determining Joints

In [23]:
# Same code as the previous one except we will be extracting landmarks
cap = cv2.VideoCapture(0)
# Setting up Mediapipe Pose Instance 
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

        #Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        #Rendering 
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(117,66,245), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows() 

[x: 0.5765835
y: 0.6187874
z: -0.9431287
visibility: 0.9997514
, x: 0.6163901
y: 0.5377107
z: -0.9088715
visibility: 0.99931014
, x: 0.63948625
y: 0.5400722
z: -0.9083818
visibility: 0.999395
, x: 0.66153646
y: 0.54357326
z: -0.90860575
visibility: 0.9992611
, x: 0.5475696
y: 0.5368048
z: -0.8963815
visibility: 0.9994943
, x: 0.52501655
y: 0.53802466
z: -0.89549553
visibility: 0.99960476
, x: 0.50478375
y: 0.54108393
z: -0.8958575
visibility: 0.99959856
, x: 0.70591
y: 0.58318686
z: -0.5403142
visibility: 0.9992575
, x: 0.4833394
y: 0.5810214
z: -0.4647326
visibility: 0.9997236
, x: 0.6243822
y: 0.70288754
z: -0.79579276
visibility: 0.9996674
, x: 0.5319315
y: 0.7098597
z: -0.7736938
visibility: 0.9997874
, x: 0.8908005
y: 0.92072046
z: -0.30044386
visibility: 0.99775356
, x: 0.29786754
y: 0.9229486
z: -0.3204085
visibility: 0.9997174
, x: 1.0431788
y: 1.3545312
z: -0.32686535
visibility: 0.16191912
, x: 0.14991948
y: 1.3300525
z: -0.54633886
visibility: 0.27097294
, x: 1.0142826
y: 1.

In [6]:
len(landmarks)

33

In [15]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk.name)
# There are 33 LandMarks in total starting from 0 to 32

NOSE
LEFT_EYE_INNER
LEFT_EYE
LEFT_EYE_OUTER
RIGHT_EYE_INNER
RIGHT_EYE
RIGHT_EYE_OUTER
LEFT_EAR
RIGHT_EAR
MOUTH_LEFT
MOUTH_RIGHT
LEFT_SHOULDER
RIGHT_SHOULDER
LEFT_ELBOW
RIGHT_ELBOW
LEFT_WRIST
RIGHT_WRIST
LEFT_PINKY
RIGHT_PINKY
LEFT_INDEX
RIGHT_INDEX
LEFT_THUMB
RIGHT_THUMB
LEFT_HIP
RIGHT_HIP
LEFT_KNEE
RIGHT_KNEE
LEFT_ANKLE
RIGHT_ANKLE
LEFT_HEEL
RIGHT_HEEL
LEFT_FOOT_INDEX
RIGHT_FOOT_INDEX


In [10]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.8513273
y: 0.9804338
z: -0.17571266
visibility: 0.99760973

In [20]:
landmarks[mp_pose.PoseLandmark.LEFT_EYE.value]

x: 0.68080103
y: 0.54397815
z: -0.6188214
visibility: 0.9999694

In [16]:
#for a bicep exercise we will need landmarks of elbow,wrist and shoulder
print("LEFT ELBOW")
print(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])
print("LEFT SHOULDER")
print(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value])
print("LEFT WRIST")
print(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

LEFT ELBOW
x: 0.9455616
y: 1.4180142
z: -0.14250772
visibility: 0.02983703

LEFT SHOULDER
x: 0.8513273
y: 0.9804338
z: -0.17571266
visibility: 0.99760973

LEFT WRIST
x: 0.8943453
y: 1.739609
z: -0.36647385
visibility: 0.010133261



CALCULATING ANGLES

In [17]:
# np.arctan2(y, x) returns the arctangent of the quotient y/x, taking into account the signs of y and x to determine the correct quadrant.
def calculate_angle(a,b,c):
    a = np.array(a) #First point
    b = np.array(b) #Mid point about which angle is found
    c = np.array(c) #End Point 
     
    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle = 360-angle
    return angle

Curl and Counter Logic

In [18]:
# Curl counter logic
""" 
if angle > 160:
    stage = "down"
if angle < 30 and stage =='down':
    stage="up"
    counter +=1
    print(counter)
"""

' \nif angle > 160:\n    stage = "down"\nif angle < 30 and stage ==\'down\':\n    stage="up"\n    counter +=1\n    print(counter)\n'

Final Code Function

In [24]:
cap = cv2.VideoCapture(0)

# Curl Counter Variables
counter = 0
stage = None

# Setting up Mediapipe Instance 
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read() #Start Capturing

        # Recolor to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detecting and storing in results list
        results = pose.process(image)

        # Recolor back to BGR
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        try:
            # Extract landmarks
            landmarks = results.pose_landmarks.landmark
            
            # Get the required coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]

            # Get the Angle
            angle = calculate_angle(shoulder,elbow,wrist)

            # Visualise 
            cv2.putText(image,str(angle),
                        tuple(np.multiply(elbow,[640,480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
            # Landmark Rendering
            mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(117,66,245), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2))
            
            # Curl Counter
            if angle > 160:
                stage = "down"
            if angle < 30 and stage == "down":
                stage="up"
                counter +=1
                print(counter)

        except: pass

        # Rendering Curl Counter 
        #Setting up Status Box
        # Set a dark color for the text
        dark_text_color = (10, 10, 10)

        # Rep data
        cv2.putText(image, 'REPS', (15, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10, 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, dark_text_color, 2, cv2.LINE_AA)
                
        # Stage data
        cv2.putText(image, 'STAGE', (65, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60, 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, dark_text_color, 2, cv2.LINE_AA)
        
        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
